In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import requests

from datetime import datetime, timedelta
from scipy.stats import norm, skew, kurtosis

In [ ]:
now = dt.datetime.now()

start = dt.datetime(now.year - 3, now.month, now.day)
end = dt.datetime(now.year, now.month, now.day)

stock_data= yf.download('MSFT', start, end)

# Calculate daily returns
returns_series = stock_data['Close'].pct_change().dropna()

# Define the confidence level (percentage)
confidence_level = 5

[*********************100%%**********************]  1 of 1 completed


**Historical VaR**

The var_historic function is designed to calculate the historic Value at Risk (VaR) for a given series of returns. VaR is a measure used in risk management to estimate the potential loss on an investment over a specified time period at a certain confidence level. The function takes a series of returns (r) and a percentage level (default is 1%) and returns the historic VaR at that specified level.

In [ ]:
def var_historic(r, confidence_level=5):
    """
    Takes in a series of returns (r), and the percentage level
(level)
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level = confidence_level)
    elif isinstance(r, pd.Series):
        return np.percentile(r, confidence_level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [ ]:
# Call the var_historic function
historic_var_result = var_historic(returns_series, confidence_level)

# Print the result
print(f"Historic Value at Risk at {confidence_level}% confidence level: {historic_var_result*100}%")
#print(f"There is a 5% chance that the daily loss will be at least {abs(historic_var_result*100):.2f}%.")

Historic Value at Risk at 5% confidence level: -2.7017046575705828%


**Parametric and Semi-Parametric VaR**

Another way to calculate VaR is to assume the set of possible outcomes behaves like a Normal (Gaussian) Distribution.

The Normal Distribution is not necessarily the best way to describe returns. However, it is a very good way to picture the concepts, and it is a good starting point for us to elaborate on more complex and realistic scenarios.
The main drawback of the parametric approach is that real world returns usually have a distribution with “fat tails” (high kurtosis).

In [ ]:
def var_gaussian(r, c_level=5, modified=False):
    """
    Returns the Parametric Gauuian VaR of a Series or DataFrame
    If "modified" is True, then the modified VaR is returned,
    using the Cornish-Fisher modification
    """
    # compute the Z score assuming it was Gaussian
    z = norm.ppf(c_level/100)
    if modified:
        # modify the Z score based on observed skewness and kurtosis
        s = skew(r)
        k = kurtosis(r)
        z = (z +
                (z**2 - 1)*s/6 +
                (z**3 -3*z)*(k-3)/24 -
                (2*z**3 - 5*z)*(s**2)/36
            )
    return (r.mean() + z*r.std(ddof=0))

In [ ]:
# Call the var_gaussian function without modification
gaussian_var_result = var_gaussian(returns_series, c_level=confidence_level, modified=False)

# Call the var_gaussian function with modification
modified_gaussian_var_result = var_gaussian(returns_series, c_level=confidence_level, modified=True)

# Print the results
print(f"Gaussian VaR at {confidence_level}% confidence level (without modification): {gaussian_var_result*100}%")
#print(f"There is a 5% chance that the daily loss will be at least {abs(gaussian_var_result*100):.2f}%.")

print(f"Gaussian VaR at {confidence_level}% confidence level (with modification): {modified_gaussian_var_result*100}%")
#print(f"There is a 5% chance that the daily loss will be at least {abs(modified_gaussian_var_result*100):.2f}%.")

Gaussian VaR at 5% confidence level (without modification): -2.7965282480357057%
Gaussian VaR at 5% confidence level (with modification): -2.8016078618944884%
